In [10]:
import pandas as pd

reddit_df = pd.read_csv("news_complete_analysis_small.csv")
reddit_df.head()

,_id,Ticker,Body,Text,url,fsai_Sentiment_score_text,fsai_Sentiment_label_text,fsai_Targeted_Sentiment,fsai_Keyword,fsai_SVO,...,lambda_concepts,lambda_analysis,lambda_doc_score.1,lambda_doc_label.1,lambda_target_sentiment.1,lambda_keywords.1,lambda_entities.1,lambda_svo.1,lambda_concepts.1,lambda_analysis.1
0,0246463d-c6ff-5f2d-8e99-0c7034f614cc,AAPL,Reese Witherspoon says career success linked t...,Your support helps us to tell the story\r\nFro...,https://www.the-independent.com/arts-entertain...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""support"", ""relevance"": 0.85}, {""tex...","[{""subject"": ""Your support"", ""verb"": ""helps"", ...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""0246463d...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""reproductive rights"", ""relevance"": ...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""0246463d..."
1,c7bc7233-1080-5f68-b373-4fd0dad2b2c5,AAPL,Motorola Edge 70 Is 5.9mm Thin But Somehow Las...,Flagship phones keep getting pricier and chunk...,https://www.yankodesign.com/2025/11/03/motorol...,0.3,Neutral,"[{""sentiment"": {""score"": 0.3, ""label"": ""Neutra...","[{""text"": ""Flagship"", ""relevance"": 0.8}, {""tex...","[{""subject"": ""Flagship phones"", ""verb"": ""keep ...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""c7bc7233...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""Flagship"", ""relevance"": 0.8}, {""tex...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""c7bc7233..."
2,920d1493-8cd9-5a58-b016-029b40d0f03d,AAPL,Google Wallet’s next update might finally fix ...,The upcoming feature was spotted in the latest...,https://www.phonearena.com/news/google-wallet-...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""Google Wallet"", ""relevance"": 0.99},...","[{""subject"": ""Google Wallet"", ""verb"": ""support...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""920d1493...",0.0,Neutral,[],"[{""text"": ""feature"", ""relevance"": 0.5}, {""text...","[{""text"": ""Google Wallet"", ""type"": ""Company""}]","[{""subject"": ""feature"", ""verb"": ""spotted"", ""ob...",[],"{""statusCode"": 200, ""body"": [{""_id"": ""920d1493..."
3,fe489e26-3d5c-5525-b15d-d0f2708df59e,AAPL,Intentions for the Week,Intentions for the Week- Week 45 in 2025\r\nHe...,https://www.shutterbean.com/2025/intentions-fo...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""Intentions"", ""relevance"": 0.5}, {""t...",[],...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""fe489e26...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""weekend"", ""relevance"": 0.1}, {""text...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""fe489e26..."
4,6c4abe93-980a-5db9-92b9-a5e9162aa737,AAPL,How 1xbet APK Download Supports Live Casino Games,Home » Articles » How 1xbet APK Download Suppo...,https://line25.com/articles/how-1xbet-apk-down...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""1xbet"", ""relevance"": 0.95}, {""text""...","[{""subject"": ""1xbet"", ""verb"": ""supports"", ""obj...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""6c4abe93...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""1xbet"", ""relevance"": 0.95}, {""text""...","[{""text"": ""1xbet"", ""type"": ""Company""}]","[{""subject"": ""1xbet"", ""verb"": ""supports"", ""obj...",[],"{""statusCode"": 200, ""body"": [{""_id"": ""6c4abe93..."


In [11]:
reddit_df['match'] = reddit_df['fsai_Sentiment_label_text'] == reddit_df['lambda_doc_label.1']

In [12]:
reddit_df['match'].sum()/reddit_df.shape[0]

0.788308009271182

In [13]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

# ---------------------------
# 1. Normalize label columns
# ---------------------------
reddit_df["actual"] = reddit_df["fsai_Sentiment_label_text"].str.lower()
reddit_df["pred"] = reddit_df["lambda_doc_label"].str.lower()

# Drop rows with missing labels
clean_df = reddit_df.dropna(subset=["actual", "pred"]).copy()

labels = ["positive", "neutral", "negative"]

# ---------------------------
# 2. Overall Accuracy
# ---------------------------
overall_accuracy = accuracy_score(clean_df["actual"], clean_df["pred"])
print("=== Overall Accuracy ===")
print(overall_accuracy)
print()

# ---------------------------
# 3. Per-Label Accuracy Breakdown
# ---------------------------
print("=== Per-Label Accuracy ===")
for label in labels:
    subset = clean_df[clean_df["actual"] == label]
    if len(subset) > 0:
        acc = accuracy_score(subset["actual"], subset["pred"])
    else:
        acc = None
    print(f"{label.capitalize():<10}: {acc} (support={len(subset)})")
print()

# ---------------------------
# 4. Precision / Recall / F1
# ---------------------------
print("=== Precision / Recall / F1 ===")
print(classification_report(clean_df["actual"], clean_df["pred"], labels=labels))
print(clean_df.shape)

# ---------------------------
# 5. Confusion Matrix
# ---------------------------
cm = confusion_matrix(clean_df["actual"], clean_df["pred"], labels=labels)

cm_df = pd.DataFrame(
    cm,
    index=[f"actual_{l}" for l in labels],
    columns=[f"pred_{l}" for l in labels]
)

print("=== Confusion Matrix ===")
print(cm_df)


=== Overall Accuracy ===
0.7866909753874203

=== Per-Label Accuracy ===
Positive  : 0.5588235294117647 (support=272)
Neutral   : 0.9491778774289985 (support=669)
Negative  : 0.48717948717948717 (support=156)

=== Precision / Recall / F1 ===
              precision    recall  f1-score   support

    positive       0.87      0.56      0.68       272
     neutral       0.76      0.95      0.85       669
    negative       0.84      0.49      0.62       156

    accuracy                           0.79      1097
   macro avg       0.82      0.67      0.71      1097
weighted avg       0.80      0.79      0.77      1097

(1097, 32)
=== Confusion Matrix ===
                 pred_positive  pred_neutral  pred_negative
actual_positive            152           119              1
actual_neutral              20           635             14
actual_negative              3            77             76


In [10]:
import pandas as pd
reddit_main_df = pd.read_csv("../sma/reddit_data.csv")
reddit_main_df.head()

,_id,Ticker,Body,Author,url,Created_at,Score,fsai_keywords,fsai_sentiment,fsai_targeted_sentiment,fsai_entities,fsai_svo,fsai_concepts
0,nmuvw5e_GOOGL,GOOGL,you should buy a real stock from a real compan...,Next_Palpitation8401,/r/wallstreetbets/comments/1omso9h/what_are_yo...,2025-11-03T10:31:22Z,10,"[{""text"": ""buy"", ""relevance"": 0.85}, {""text"": ...","{""document"": {""score"": 0.1, ""label"": ""Neutral""}}","[{""text"": ""GOOGL"", ""sentiment"": {""score"": 0.3,...","[{""text"": ""GOOG"", ""type"": ""Company""}, {""text"":...","[{""subject"": ""you"", ""verb"": ""should buy"", ""obj...",[]
1,nmt2zd3_GOOGL,GOOGL,Cheaper than AMZN GOOGL?,rankiba,/r/wallstreetbets/comments/1om5mmo/what_do_we_...,2025-11-03T01:31:08Z,1,"[{""text"": ""Cheaper"", ""relevance"": 0.85}, {""tex...","{""document"": {""score"": 0.3, ""label"": ""Positive""}}","[{""text"": ""GOOGL"", ""sentiment"": {""score"": 0.3,...","[{""text"": ""AMZN"", ""type"": ""Company""}, {""text"":...","[{""subject"": ""GOOGL"", ""verb"": ""is"", ""object"": ...",[]
2,nmxnz9k_F,F,"The full answer is complicated, as it requires...",shinobinc,/r/wallstreetbets/comments/1omm7xs/poet_the_ph...,2025-11-03T19:49:48Z,1,"[{""text"": ""manufacturing"", ""relevance"": 0.85},...","{""document"": {""score"": 0.1, ""label"": ""Neutral""}}","[{""text"": ""F"", ""sentiment"": {""score"": 0.0, ""la...","[{""text"": ""POET"", ""type"": ""Company""}, {""text"":...","[{""subject"": ""POET"", ""verb"": ""has"", ""object"": ...",[]
3,nmxrhv5_F,F,Anyone thinking Fabrinet $FN will drop? Up 20%...,OmegaThree3,/r/wallstreetbets/comments/1okv6l7/weekly_earn...,2025-11-03T20:06:48Z,5,"[{""text"": ""drop"", ""relevance"": 0.85}, {""text"":...","{""document"": {""score"": -0.1, ""label"": ""Neutral""}}","[{""text"": ""F"", ""sentiment"": {""score"": -0.1, ""l...","[{""text"": ""Fabrinet"", ""type"": ""Company""}, {""te...","[{""subject"": ""Anyone"", ""verb"": ""thinking"", ""ob...",[]
4,nmss8lb_AMZN,AMZN,seagate + amazon,Either-Lie-9000,/r/wallstreetbets/comments/1omso9h/what_are_yo...,2025-11-03T00:26:08Z,6,"[{""text"": ""seagate"", ""relevance"": 0.5}, {""text...","{""document"": {""score"": 0.0, ""label"": ""Neutral""}}","[{""text"": ""AMZN"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""seagate"", ""type"": ""Company""}, {""tex...","[{""subject"": ""seagate"", ""verb"": ""partner"", ""ob...",[]


In [12]:
reddit_main_df.fsai_entities.value_counts()

fsai_entities
[{"text": "META", "type": "Company"}]                                                                                         122
[{"text": "PLTR", "type": "Company"}]                                                                                          95
[{"text": "AMD", "type": "Company"}]                                                                                           75
[{"text": "NVDA", "type": "Company"}]                                                                                          48
[{"text": "RDDT", "type": "Company"}]                                                                                          32
                                                                                                                             ... 
[{"text": "TSLA", "type": "Company"}, {"text": "NVDA", "type": "Company"}, {"text": "IBIT", "type": "Company"}]                 1
[{"text": "Michael Burry", "type": "Person"}, {"text": "Elon", "type": "Pers

In [14]:
reddit_small_df = pd.read_csv("reddit_complete_analysis.csv")
reddit_small_df.head()

,Unnamed: 0,_id,Ticker,Body,Text,url,fsai_keywords,sentiment_score,sentiment_label,target_sentiment,lambda_doc_score,lambda_doc_label,lambda_target_sentiment,lambda_keywords,lambda_analysis
0,0,nmuvw5e_GOOGL,GOOGL,you should buy a real stock from a real compan...,NaN,/r/wallstreetbets/comments/1omso9h/what_are_yo...,"[{""text"": ""buy"", ""relevance"": 0.85}, {""text"": ...",0.1,Neutral,"[{""sentiment"": {""score"": 0.3, ""label"": ""Positi...",0.3,Positive,"[{""text"": ""GOOGL"", ""sentiment"": {""score"": 0.5,...","[{""text"": ""buy"", ""relevance"": 0.85}, {""text"": ...","{""statusCode"": 200, ""body"": [{""sentiment"": {""d..."
1,1,nmt2zd3_GOOGL,GOOGL,Cheaper than AMZN GOOGL?,NaN,/r/wallstreetbets/comments/1om5mmo/what_do_we_...,"[{""text"": ""Cheaper"", ""relevance"": 0.85}, {""tex...",0.3,Positive,"[{""sentiment"": {""score"": 0.3, ""label"": ""Positi...",0.0,Neutral,"[{""text"": ""GOOGL"", ""sentiment"": {""score"": 0.0,...","[{""text"": ""Cheaper"", ""relevance"": 0.85}, {""tex...","{""statusCode"": 200, ""body"": [{""sentiment"": {""d..."
2,2,nmss8lb_AMZN,AMZN,seagate + amazon,NaN,/r/wallstreetbets/comments/1omso9h/what_are_yo...,"[{""text"": ""seagate"", ""relevance"": 0.5}, {""text...",0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...",0.0,Neutral,"[{""text"": ""AMZN"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""seagate"", ""relevance"": 0.5}, {""text...","{""statusCode"": 200, ""body"": [{""sentiment"": {""d..."
3,3,nmuvw5e_AMZN,AMZN,you should buy a real stock from a real compan...,NaN,/r/wallstreetbets/comments/1omso9h/what_are_yo...,"[{""text"": ""buy"", ""relevance"": 0.85}, {""text"": ...",0.3,Positive,"[{""sentiment"": {""score"": 0.5, ""label"": ""Positi...",0.3,Positive,"[{""text"": ""AMZN"", ""sentiment"": {""score"": 0.5, ...","[{""text"": ""buy"", ""relevance"": 0.85}, {""text"": ...","{""statusCode"": 200, ""body"": [{""sentiment"": {""d..."
4,4,nmv0sk7_AMZN,AMZN,Iren is up 19% premarket…. \nNbis I have 150k...,NaN,/r/wallstreetbets/comments/1omso9h/what_are_yo...,"[{""text"": ""premarket"", ""relevance"": 0.8}, {""te...",0.5,Positive,"[{""sentiment"": {""score"": 0.5, ""label"": ""Positi...",0.5,Positive,"[{""text"": ""AMZN"", ""sentiment"": {""score"": 0.5, ...","[{""text"": ""premarket"", ""relevance"": 0.8}, {""te...","{""statusCode"": 200, ""body"": [{""sentiment"": {""d..."


In [15]:
reddit_df_to_send = pd.merge(reddit_main_df, reddit_small_df, on="_id", how="left")
reddit_df_to_send.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1000 non-null   object 
 1   Ticker_x                 1000 non-null   object 
 2   Body_x                   1000 non-null   object 
 3   Author                   1000 non-null   object 
 4   url_x                    1000 non-null   object 
 5   Created_at               1000 non-null   object 
 6   Score                    1000 non-null   int64  
 7   fsai_keywords_x          1000 non-null   object 
 8   fsai_sentiment           1000 non-null   object 
 9   fsai_targeted_sentiment  1000 non-null   object 
 10  fsai_entities            1000 non-null   object 
 11  fsai_svo                 1000 non-null   object 
 12  fsai_concepts            1000 non-null   object 
 13  Unnamed: 0               1000 non-null   int64  
 14  Ticker_y                 

In [17]:
reddit_df_to_send.to_csv("reddit_complete_for_aditi.csv")

In [18]:
import pandas as pd
news_main_df = pd.read_csv("../sma/news_fsai_export.csv")
news_main_df.head()

,_id,Ticker,Body,Text,Author,Host,url,Created_at,market_time_news,fsai_Sentiment_score_text,fsai_Sentiment_label_text,fsai_Targeted_Sentiment,fsai_Keyword,fsai_SVO,fsai_Company,fsai_Person,fsai_Concept,fsai_Categories
0,0246463d-c6ff-5f2d-8e99-0c7034f614cc,AAPL,Reese Witherspoon says career success linked t...,Your support helps us to tell the story\r\nFro...,Hannah Roberts,The-independent.com,https://www.the-independent.com/arts-entertain...,2025-11-03T09:52:29Z,2025-11-03T09:00:00Z,0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""support"", ""relevance"": 0.85}, {""tex...","[{""subject"": ""Your support"", ""verb"": ""helps"", ...",[],[],[],[]
1,c7bc7233-1080-5f68-b373-4fd0dad2b2c5,AAPL,Motorola Edge 70 Is 5.9mm Thin But Somehow Las...,Flagship phones keep getting pricier and chunk...,JC Torres,Yanko Design,https://www.yankodesign.com/2025/11/03/motorol...,2025-11-03T09:45:03Z,2025-11-03T09:00:00Z,0.3,Neutral,"[{""text"": ""Motorola"", ""sentiment"": {""score"": 0...","[{""text"": ""Flagship"", ""relevance"": 0.8}, {""tex...","[{""subject"": ""Flagship phones"", ""verb"": ""keep ...","[{""text"": ""Motorola"", ""type"": ""Company""}]",[],[],[]
2,920d1493-8cd9-5a58-b016-029b40d0f03d,AAPL,Google Wallet’s next update might finally fix ...,The upcoming feature was spotted in the latest...,Tsveta Ermenkova,PhoneArena,https://www.phonearena.com/news/google-wallet-...,2025-11-03T09:41:37Z,2025-11-03T09:00:00Z,0.0,Neutral,"[{""text"": ""Google Wallet"", ""sentiment"": {""scor...","[{""text"": ""Google Wallet"", ""relevance"": 0.99},...","[{""subject"": ""Google Wallet"", ""verb"": ""support...","[{""text"": ""Google Wallet"", ""type"": ""Company""}]",[],[],[]
3,fe489e26-3d5c-5525-b15d-d0f2708df59e,AAPL,Intentions for the Week,Intentions for the Week- Week 45 in 2025\r\nHe...,NaN,Shutterbean.com,https://www.shutterbean.com/2025/intentions-fo...,2025-11-03T09:22:18Z,2025-11-03T09:00:00Z,0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""Intentions"", ""relevance"": 0.5}, {""t...",[],[],[],[],[]
4,6c4abe93-980a-5db9-92b9-a5e9162aa737,AAPL,How 1xbet APK Download Supports Live Casino Games,Home » Articles » How 1xbet APK Download Suppo...,Kate Dagli,Line25.com,https://line25.com/articles/how-1xbet-apk-down...,2025-11-03T09:16:25Z,2025-11-03T09:00:00Z,0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""1xbet"", ""relevance"": 0.95}, {""text""...","[{""subject"": ""1xbet"", ""verb"": ""supports"", ""obj...","[{""text"": ""1xbet"", ""type"": ""Company""}]",[],[],[]


In [20]:
news_small_df = pd.read_csv("news_complete_analysis_small.csv")
news_small_df.head()

,_id,Ticker,Body,Text,url,fsai_Sentiment_score_text,fsai_Sentiment_label_text,fsai_Targeted_Sentiment,fsai_Keyword,fsai_SVO,...,lambda_concepts,lambda_analysis,lambda_doc_score.1,lambda_doc_label.1,lambda_target_sentiment.1,lambda_keywords.1,lambda_entities.1,lambda_svo.1,lambda_concepts.1,lambda_analysis.1
0,0246463d-c6ff-5f2d-8e99-0c7034f614cc,AAPL,Reese Witherspoon says career success linked t...,Your support helps us to tell the story\r\nFro...,https://www.the-independent.com/arts-entertain...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""support"", ""relevance"": 0.85}, {""tex...","[{""subject"": ""Your support"", ""verb"": ""helps"", ...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""0246463d...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""reproductive rights"", ""relevance"": ...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""0246463d..."
1,c7bc7233-1080-5f68-b373-4fd0dad2b2c5,AAPL,Motorola Edge 70 Is 5.9mm Thin But Somehow Las...,Flagship phones keep getting pricier and chunk...,https://www.yankodesign.com/2025/11/03/motorol...,0.3,Neutral,"[{""sentiment"": {""score"": 0.3, ""label"": ""Neutra...","[{""text"": ""Flagship"", ""relevance"": 0.8}, {""tex...","[{""subject"": ""Flagship phones"", ""verb"": ""keep ...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""c7bc7233...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""Flagship"", ""relevance"": 0.8}, {""tex...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""c7bc7233..."
2,920d1493-8cd9-5a58-b016-029b40d0f03d,AAPL,Google Wallet’s next update might finally fix ...,The upcoming feature was spotted in the latest...,https://www.phonearena.com/news/google-wallet-...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""Google Wallet"", ""relevance"": 0.99},...","[{""subject"": ""Google Wallet"", ""verb"": ""support...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""920d1493...",0.0,Neutral,[],"[{""text"": ""feature"", ""relevance"": 0.5}, {""text...","[{""text"": ""Google Wallet"", ""type"": ""Company""}]","[{""subject"": ""feature"", ""verb"": ""spotted"", ""ob...",[],"{""statusCode"": 200, ""body"": [{""_id"": ""920d1493..."
3,fe489e26-3d5c-5525-b15d-d0f2708df59e,AAPL,Intentions for the Week,Intentions for the Week- Week 45 in 2025\r\nHe...,https://www.shutterbean.com/2025/intentions-fo...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""Intentions"", ""relevance"": 0.5}, {""t...",[],...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""fe489e26...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""weekend"", ""relevance"": 0.1}, {""text...",[],[],[],"{""statusCode"": 200, ""body"": [{""_id"": ""fe489e26..."
4,6c4abe93-980a-5db9-92b9-a5e9162aa737,AAPL,How 1xbet APK Download Supports Live Casino Games,Home » Articles » How 1xbet APK Download Suppo...,https://line25.com/articles/how-1xbet-apk-down...,0.0,Neutral,"[{""sentiment"": {""score"": 0.0, ""label"": ""Neutra...","[{""text"": ""1xbet"", ""relevance"": 0.95}, {""text""...","[{""subject"": ""1xbet"", ""verb"": ""supports"", ""obj...",...,[],"{""statusCode"": 200, ""body"": [{""_id"": ""6c4abe93...",0.0,Neutral,"[{""text"": ""AAPL"", ""sentiment"": {""score"": 0.0, ...","[{""text"": ""1xbet"", ""relevance"": 0.95}, {""text""...","[{""text"": ""1xbet"", ""type"": ""Company""}]","[{""subject"": ""1xbet"", ""verb"": ""supports"", ""obj...",[],"{""statusCode"": 200, ""body"": [{""_id"": ""6c4abe93..."
